In [19]:
import pandas as pd
import numpy as np
from datetime import date
import warnings
warnings.filterwarnings('ignore') 

In [20]:
df= pd.read_csv("bikes.csv", index_col=0)
df.head(2)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801


## Empezamos el EDA
Unificamos y cambiamos los nombres de las columnas

In [21]:
columnas={"instant":"index","dteday":"date","season":"season","yr":"year","mnth":"month","holiday":"holiday",\
    "weekday":"weeakday","workingday":"workingday","weathersit":"weathersit","temp":"temperature","atemp":"sensation",\
    "hum":"humidity","windspeed":"windspeed","casual":"casual","registered":"registered","cnt":"count"}

In [22]:
df.rename(columns = columnas, inplace = True)
df.head(2)

,index,date,season,year,month,holiday,weeakday,workingday,weathersit,temperature,sensation,humidity,windspeed,casual,registered,count
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801


In [23]:
print(f'Tenemos {df.shape[0]} filas y {df.shape[1]} columnas')

Tenemos 730 filas y 16 columnas


Miramos nuestros duplicados

In [24]:
df.duplicated().sum()

0

Verificamos los nulos

In [25]:
df.isnull().sum()

index          0
date           0
season         0
year           0
month          0
holiday        0
weeakday       0
workingday     0
weathersit     0
temperature    0
sensation      0
humidity       0
windspeed      0
casual         0
registered     0
count          0
dtype: int64

Genial! No tenemos nulos

Sacamos los principales estadisticos

In [26]:
df.describe(include="object").T

,count,unique,top,freq
date,730,730,01-01-2018,1
season,730,4,autumn,188


In [27]:
df.describe(include="number").T

,count,mean,std,min,25%,50%,75%,max
index,730.0,365.500000,210.877136,1.000000,183.250000,365.500000,547.750000,730.000000
year,730.0,0.500000,0.500343,0.000000,0.000000,0.500000,1.000000,1.000000
month,730.0,6.526027,3.450215,1.000000,4.000000,7.000000,10.000000,12.000000
holiday,730.0,0.028767,0.167266,0.000000,0.000000,0.000000,0.000000,1.000000
weeakday,730.0,2.997260,2.006161,0.000000,1.000000,3.000000,5.000000,6.000000
workingday,730.0,0.683562,0.465405,0.000000,0.000000,1.000000,1.000000,1.000000
weathersit,730.0,1.394521,0.544807,1.000000,1.000000,1.000000,2.000000,3.000000
temperature,730.0,20.319259,7.506729,2.424346,13.811885,20.465826,26.880615,35.328347
sensation,730.0,23.726322,8.150308,3.953480,16.889713,24.368225,30.445775,42.044800
humidity,730.0,62.765175,14.237589,0.000000,52.000000,62.625000,72.989575,97.250000


In [28]:
df.dtypes

index            int64
date            object
season          object
year             int64
month            int64
holiday          int64
weeakday         int64
workingday       int64
weathersit       int64
temperature    float64
sensation      float64
humidity       float64
windspeed      float64
casual           int64
registered       int64
count            int64
dtype: object

## Limpieza

Como vemos tenemos columnas con el tipo de dato mal, algunas están como númericas pero realmente son categoricas. Así que vamos a cambiarlas.

In [29]:
df["year"]=df["year"].astype("category", errors="ignore")
df["month"]=df["month"].astype("object", errors="ignore")
df["holiday"]=df["holiday"].astype("object", errors="ignore")
#df["weeakday"]=df["weeakday"].astype("category", errors="ignore")
df["workingday"]=df["workingday"].astype("object", errors="ignore")
#df["weathersit"]=df["weathersit"].astype("category", errors="ignore")

Convertimos la columna "date" a tipo de dato Date

In [30]:
df["date"]=pd.to_datetime(df['date'], format= "%d-%m-%Y")
df.sort_values(by='date', inplace=True)
#hacemos un reset
df.reset_index(drop=True, inplace=True)

In [31]:
# Verificamos
df.dtypes

index                   int64
date           datetime64[ns]
season                 object
year                 category
month                  object
holiday                object
weeakday                int64
workingday             object
weathersit              int64
temperature           float64
sensation             float64
humidity              float64
windspeed             float64
casual                  int64
registered              int64
count                   int64
dtype: object

La columna de estación está mal, ya que no coinciden nuestras fechas ya que están invertidas vamos a hacer un mapeo para ponerlo en el orden que corresponde.

Vemos que las categorias de season estan mal asignadas --> hacemos un mapeo:

    - spring --> winter
    - summer --> spring
    - autumn --> summer
    - winter --> autumn

In [32]:
mapa = {'spring':'winter','summer':'spring','autumn':'summer','winter':'autumn'}
df['season'] = df['season'].map(mapa)
df.sample(2)

,index,date,season,year,month,holiday,weeakday,workingday,weathersit,temperature,sensation,humidity,windspeed,casual,registered,count
115,116,2018-04-26,spring,0,4,0,2,1,1,25.898347,29.70415,72.9167,21.875500,678,3722,4400
450,451,2019-03-27,spring,1,3,0,2,1,1,13.256653,15.78270,29.0000,12.541864,531,4571,5102


Vemos que algunos meses no están bien asignados. Vamos a reemplazarlos por los valores correctos:

In [33]:
def cambiar_month(col):
    return col.month

In [34]:
df['month'] = df['date'].apply(cambiar_month)
df.sample(2)

,index,date,season,year,month,holiday,weeakday,workingday,weathersit,temperature,sensation,humidity,windspeed,casual,registered,count
222,223,2018-08-11,summer,0,8,0,4,1,1,29.417500,32.57605,42.375,11.041332,812,3980,4792
514,515,2019-05-30,spring,1,5,0,3,1,2,26.923347,30.55645,67.000,9.000043,745,6110,6855


### Cambiamos el weekday (está mal)

In [35]:
mapa_dias={6:1,0:2,1:3,2:4,3:5,4:6,5:7}
df['weeakday']=df['weeakday'].map(mapa_dias)

In [36]:
# Cambiamos holiday
def hol(x):
    festivos = pd.DatetimeIndex(['2018-1-1', '2018-1-15', '2018-2-19','2018-4-16','2018-5-28','2018-7-4',
                                 '2018-9-3','2018-10-8','2018-11-12','2018-11-22','2018-12-25','2019-1-1', 
                                 '2019-1-21', '2019-2-18','2019-4-16','2019-5-27','2019-7-4','2019-9-2',
                                 '2019-10-14','2019-11-11','2019-11-28','2019-12-25'])
    if pd.DatetimeIndex([x]).isin(festivos):
        return 1 
    else:
        return 0

In [37]:
df['holiday'] = df['date'].apply(hol)
df['holiday'].sum()

22

Eliminamos las columnas redundantes.

In [38]:
df.drop(["workingday","temperature","casual","registered"], axis=1, inplace=True)
df.head(2)

,index,date,season,year,month,holiday,weeakday,weathersit,sensation,humidity,windspeed,count
0,1,2018-01-01,winter,0,1,1,1,2,18.18125,80.5833,10.749882,985
1,2,2018-01-02,winter,0,1,0,2,2,17.68695,69.6087,16.652113,801


guardamos el dataframe

In [40]:
df.to_csv("bikes_limpio.csv")